# Travelio Challenge


Data Preparation

In [1]:
import os
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
dataset_path = "CHALLNGE_DATA-SET/"

# Parameters
img_height, img_width = 224, 224  # Consistent size for all images
batch_size = 32

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split data into training and validation sets
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


2024-07-12 17:43:32.790123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 302 images belonging to 7 classes.
Found 73 images belonging to 7 classes.


# Model Creation

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Load the VGG16 model with pre-trained weights, exclude top layers
base_model = VGG16(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Create the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)

# Training and Validation

In [3]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10  # You can increase the number of epochs
)


Epoch 1/10


10/10 [==============================] - 189s 19s/step - loss: 4.2601 - accuracy: 0.3311 - val_loss: 0.9093 - val_accuracy: 0.5890
Epoch 2/10
10/10 [==============================] - 129s 13s/step - loss: 1.2602 - accuracy: 0.6457 - val_loss: 0.5906 - val_accuracy: 0.7808
Epoch 3/10
10/10 [==============================] - 127s 13s/step - loss: 0.5189 - accuracy: 0.8377 - val_loss: 0.4260 - val_accuracy: 0.8356
Epoch 4/10
10/10 [==============================] - 127s 13s/step - loss: 0.3304 - accuracy: 0.8742 - val_loss: 0.4084 - val_accuracy: 0.8356
Epoch 5/10
10/10 [==============================] - 127s 13s/step - loss: 0.2311 - accuracy: 0.9172 - val_loss: 0.4015 - val_accuracy: 0.8356
Epoch 6/10
10/10 [==============================] - 128s 13s/step - loss: 0.1550 - accuracy: 0.9503 - val_loss: 0.4193 - val_accuracy: 0.8219
Epoch 7/10
10/10 [==============================] - 127s 13s/step - loss: 0.1314 - accuracy: 0.9570 - val_loss: 0.2778 - val_accuracy: 0.9452
Epoc

#  Save the Model

In [4]:
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)


In [5]:
print(tf.__version__)

2.15.0


In [6]:
model.save('attraction_model.h5')

d:\Travelio ML v2\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Load and Use the Model for Prediction

Training

In [8]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the model
model1 = load_model('attraction_model.h5')

# Function to predict the class of a single image
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = list(train_generator.class_indices.keys())[class_idx]

    return class_label

# Predict an image
img_path = './Test pics/khofo_edit.jpg'
predicted_class = predict_image(img_path, model1)
print(f'The image is predicted to be: {predicted_class}')


1/1 [==============================] - 1s 910ms/step
The image is predicted to be: GIZA_PYRAMIDS
